all retrievers

https://python.langchain.com/docs/integrations/retrievers/



https://huggingface.co/BAAI/bge-reranker-v2-m3

In [ ]:
# 使用 bge-reranker
!pip install -U FlagEmbedding

# FlagEmbedding 是什麼？

In [1]:
# 在 jupyter 中顯示進度條功能
# ! pip install ipywidgets
# ! jupyter labextension install @jupyter-widgets/jupyterlab-manager
# ! jupyter nbextension enable --py widgetsnbextension --sys-prefix

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 1.8 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 kB 5.1 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 10.6 MB/s eta 0:00:00a 0:00:01


In [1]:
import ipywidgets as widgets
widgets.IntSlider()

IntSlider(value=0)

In [2]:
! jupyter nbextension enable --py widgetsnbextension --sys-prefix

usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert run server troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [1]:
import torch
print(torch.cuda.is_available())

True


In [1]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-v2-m3").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

query = "query"
passage = "passage"

inputs = tokenizer(query, passage, return_tensors="pt", padding=True, truncation=True).to("cuda")

# with torch.no_grad():
#     logits = model(**inputs).logits
#     print(logits)  # ¨S crash ¤~·|Åã¥Ü


tensor([[-5.6501]], device='cuda:0')


In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import torch.nn.functional as F  # ¥Î©ó sigmoid


# ¸ü¤J¼Ò«¬»P tokenizer
model = AutoModelForSequenceClassification.from_pretrained("BAAI/bge-reranker-v2-m3").to("cuda")
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-reranker-v2-m3")

# ¬d¸ß»P¤å¥ó
query = "what is panda?"
passage = "The giant panda is native to China."

# ½s½X¨ÃÂà¨ì GPU
inputs = tokenizer(query, passage, return_tensors="pt", padding=True, truncation=True, max_length=512).to("cuda")

# ¤£°µ¤Ï¦V¶Ç¼½
with torch.no_grad():
    logits = model(**inputs).logits  # shape: [1, 1]
    score = logits[0].item()  # ­ì©l¤À¼Æ
    normalized_score = torch.sigmoid(logits)[0].item()  # 0~1 ½d³ò

# Åã¥Üµ²ªG
print(f"Raw score: {score}")
print(f"Normalized score (sigmoid): {normalized_score}")

Raw score: -2.4507484436035156
Normalized score (sigmoid): 0.07938383519649506


In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score) # -5.65234375

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
score = reranker.compute_score(['query', 'passage'], normalize=True)
print(score) # 0.003497010252573502

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores) # [-8.1875, 5.26171875]

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], normalize=True)
print(scores) # [0.00027803096387751553, 0.9948403768236574]


In [ ]:
from FlagEmbedding import FlagLLMReranker
reranker = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
# reranker = FlagLLMReranker('BAAI/bge-reranker-v2-gemma', use_bf16=True) # You can also set use_bf16=True to speed up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores)


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from FlagEmbedding import LayerWiseFlagLLMReranker
reranker = LayerWiseFlagLLMReranker('BAAI/bge-reranker-v2-minicpm-layerwise', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
# reranker = LayerWiseFlagLLMReranker('BAAI/bge-reranker-v2-minicpm-layerwise', use_bf16=True) # You can also set use_bf16=True to speed up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'], cutoff_layers=[28]) # Adjusting 'cutoff_layers' to pick which layers are used for computing the score.
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], cutoff_layers=[28])
print(scores)



You are using a model of type xlm-roberta to instantiate a model of type minicpm. This is not supported for all configurations of models and can yield errors.
LayerWiseMiniCPMForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly defined. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update i

In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation
# reranker = LayerWiseFlagLLMReranker('BAAI/bge-reranker-v2-minicpm-layerwise', use_bf16=True) # You can also set use_bf16=True to speed up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'], cutoff_layers=[28]) # Adjusting 'cutoff_layers' to pick which layers are used for computing the score.
print(score)

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], cutoff_layers=[28])
print(scores)


In [ ]:
score = reranker.compute_score(['query', 'passage'])


In [ ]:
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-base', use_fp16=False, device="cuda")
score = reranker.compute_score(['query', 'passage'])
print(next(reranker.model.parameters()).device)

In [ ]:
# kernel will die
from FlagEmbedding import FlagReranker
reranker = FlagReranker('BAAI/bge-reranker-v2-m3', use_fp16=True) # Setting use_fp16 to True speeds up computation with a slight performance degradation

score = reranker.compute_score(['query', 'passage'])
print(score) # -5.65234375

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
score = reranker.compute_score(['query', 'passage'], normalize=True)
print(score) # 0.003497010252573502

scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']])
print(scores) # [-8.1875, 5.26171875]

# You can map the scores into 0-1 by set "normalize=True", which will apply sigmoid function to the score
scores = reranker.compute_score([['what is panda?', 'hi'], ['what is panda?', 'The giant panda (Ailuropoda melanoleuca), sometimes called a panda bear or simply panda, is a bear species endemic to China.']], normalize=True)
print(scores) # [0.00027803096387751553, 0.9948403768236574]


In [5]:
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain.retrievers import ContextualCompressionRetriever

In [7]:
# ªì©l¤Æ CrossEncoder¡]¹w³]¨Ï¥Î BGE reranker¡^
model = HuggingFaceCrossEncoder(model_name="BAAI/bge-reranker-base")

# ¥]¸Ë CrossEncoderReranker
compressor = CrossEncoderReranker(model=model, top_n=5)

# °²³]§A¤w¦³ base_retriever¡]¨Ò¦p FAISS retriever¡^
compression_retriever = ContextualCompressionRetriever(
    base_retriever=base_retriever,
    base_compressor=compressor
)

# ¬d¸ß rerank µ²ªG
docs = compression_retriever.invoke("查詢內容：")
for i, doc in enumerate(docs, 1):
    print(f"{i}. {doc.page_content}")

/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'base_retriever' is not defined

In [6]:
from langchain_community.retrievers import BM25Retriever # retriver
from langchain_core.documents import Document
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader('../data/PDF_file.pdf')
docs = loader.load_and_split()





In [7]:
bm25_retriever = BM25Retriever.from_documents(docs)
bm25_retriever.k = 10  # 回傳前 10 篇最相關的段落

ImportError: Could not import rank_bm25, please install with `pip install rank_bm25`.

In [ ]:
# 也可以將 doc 切分成更細的 chunk 再丟給 bm25_retriever
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

split_docs = text_splitter.split_documents(docs)  # ³o¸Ì¶Ç¤Jªº¬O¤@°ï page-based document

bm25_retriever = BM25Retriever.from_documents(split_docs)

